In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

tạo connector tới mysql

In [6]:
host='localhost'
database='web_project2'
user='root'
password='30032001'
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

In [7]:
query = 'SELECT * FROM web_project2.movie_movie_genres'
movie_genre_data = pd.read_sql(query, engine)
print(movie_genre_data)


          id  movie_id  genre_id
0          1         2        18
1          2         2        35
2          4         2        80
3          3         2     10749
4          5         3        18
...      ...       ...       ...
13230   9502   1232943        35
13231   9503   1232943     10749
13232   9939   1241742        18
13233   9991   1241743        18
13234  18678   1242810         1

[13235 rows x 3 columns]


In [8]:
pivot_movie_genre = movie_genre_data.pivot_table(index=['movie_id'], columns=['genre_id'], values=['id'], fill_value=0)
pivot_movie_genre = pd.DataFrame(np.where(pivot_movie_genre != 0, 1, pivot_movie_genre),
                                    index=pivot_movie_genre.index,
                                    columns=pivot_movie_genre.columns)

lấy dữ liệu genre cho movie

lọc các movie chưa được rate lần nào cho bảng genre của movie

In [9]:
ratings = pd.read_csv('ratings_small.csv')

In [10]:
rated_movies = set(ratings['movieId'])
rated_movie_genre = pivot_movie_genre[pivot_movie_genre.index.isin(rated_movies)]
rated_movie_genre['movie_id'] = rated_movie_genre.index
rated_movie_genre.reset_index(drop=True, inplace=True)


C:\Users\Dang Dung\AppData\Local\Temp\ipykernel_14912\3091134729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_movie_genre['movie_id'] = rated_movie_genre.index


In [11]:
ratings = ratings[ratings['movieId'].isin(rated_movie_genre['movie_id'])]
ratings = ratings.drop(columns=['timestamp'])

In [39]:
ratings

,userId,movieId,rating
5,1,1263,2.0
10,1,1371,2.5
11,1,1405,1.0
13,1,2105,4.0
15,1,2193,2.0
...,...,...,...
99992,671,5816,4.0
99993,671,5902,3.5
99996,671,5991,4.5
99997,671,5995,4.0


In [40]:
rated_movie_genre

id                                               ...            \
genre_id    1   12   14   16   18   27   28   35   36   37  ...   80   99   
0         0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  0.0   
1         0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0   
2         0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0   
3         0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  1.0  0.0   
4         0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3901      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0   
3902      0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0   
3903      0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  ...  0.0  0.0   
3904      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3905      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

                                                 movie_id  
genre_id  878 9648 10402 10749 10751 10752 10770           
0         0.0  0.0   0.0   1.0   0.0   0.0   0.0        2  
1         0.0  0.0   0.0   1.0   0.0   0.0   0.0        3  
2         0.0  0.0   0.0   0.0   0.0   0.0   0.0        5  
3         0.0  0.0   0.0   0.0   0.0   0.0   0.0        6  
4         1.0  0.0   0.0   0.0   0.0   0.0   0.0       11  
...       ...  ...   ...   ...   ...   ...   ...      ...  
3901      0.0  0.0   0.0   0.0   0.0   0.0   0.0   158238  
3902      0.0  0.0   0.0   0.0   0.0   0.0   0.0   160080  
3903      0.0  0.0   0.0   0.0   0.0   0.0   1.0   160718  
3904      0.0  0.0   0.0   0.0   1.0   0.0   1.0   161594  
3905      0.0  0.0   0.0   0.0   0.0   0.0   1.0   161830  

[3906 rows x 21 columns]

we got rated movie_genre and ratings 

# training

In [12]:
X_train_counts = rated_movie_genre.to_numpy()
X_train_counts[:,:20]
# movie x genre

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

tạo tệp test vs train theo dữ liệu ratings

In [13]:
ratings.head()

,userId,movieId,rating
5,1,1263,2.0
10,1,1371,2.5
11,1,1405,1.0
13,1,2105,4.0
15,1,2193,2.0


In [14]:
ratings_train, ratings_test = train_test_split(ratings, test_size=0.1, random_state=40)

# build model

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

In [16]:
tfidf.shape

(3906, 21)

In [17]:
def get_items_rated_by_user(rate_matrix, user_id):
    ids = rate_matrix.loc[rate_matrix['userId'] == user_id].index
    item_ids = rate_matrix.loc[ids, 'movieId']
    # tim thu tu cua movieId do trong bang movie_genre
    item_order = rated_movie_genre[rated_movie_genre['movie_id'].isin(item_ids)].index
    scores = rate_matrix.loc[ids, 'rating']
    return (item_order, scores)
    # suy ra duoc genres cua movie do duoc user rated bao nhieu 

In [18]:
# đi tìm các hệ số của Ridge Regression cho mỗi user:
n_users = len(set(ratings['userId']))
print(len(set(ratings['userId'])))

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))
i = 0

for user_id in set(ratings['userId']):    
    ids, scores = get_items_rated_by_user(ratings_train, user_id)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores) 
    W[:, i] = clf.coef_
    b[0, i] = clf.intercept_
    i += 1 
    # tuc la moi i dai dien cho mot user_id nhung theo so thu tu vi neu dung id thi se loan va table se rat lon. 
    # vi du user_id 100 se la i = 1, user_id 200 se la i = 2 va lat nua xuong duoi evaluate se dung i thay vi use_id 
    # vi dung in set(ratings['userId']) nen se khong so sai thu tu 

Yhat = tfidf.dot(W) + b

671


In [19]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    i = 0 
    for user_id in set(ratings['userId']):
        ids, scores_truth = get_items_rated_by_user(rates, user_id)
        scores_pred = Yhat[ids, i]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
        i += 1
        # i chinh la user_id nhung o dang so thu tu 
        
    return np.sqrt(se/cnt)

In [20]:
evaluate(Yhat, ratings_train, W, b)


0.9390717834065421

In [21]:
evaluate(Yhat, ratings_test, W, b)

0.9685785668589612

evaluation content-based 

In [22]:
print ('RMSE for training:', evaluate(Yhat, ratings_train, W, b))
print ('RMSE for test    :', evaluate(Yhat, ratings_test, W, b))
#RMSE for training: 0.9390717834065421
#RMSE for test    : 0.9685785668589612

RMSE for training: 0.9390717834065421
RMSE for test    : 0.9685785668589612


Filtering collaboration

In [23]:
ratings

,userId,movieId,rating
5,1,1263,2.0
10,1,1371,2.5
11,1,1405,1.0
13,1,2105,4.0
15,1,2193,2.0
...,...,...,...
99992,671,5816,4.0
99993,671,5902,3.5
99996,671,5991,4.5
99997,671,5995,4.0


In [542]:
#user-based
ratings_pivot = ratings.pivot(index=['userId'], columns=['movieId'], values='rating')
ratings_pivot.fillna(0, inplace=True)
user_movie_rating = scaler.fit_transform(ratings_pivot)

user_similarity = cosine_similarity(user_movie_rating)

temp1 = user_similarity.dot(user_movie_rating)
user_based_predict = (temp1/np.column_stack([user_similarity.sum(axis=1)] * temp1.shape[-1])).T


In [543]:
user_movie_rating.T.shape

(3906, 671)

In [544]:
user_based_predict.shape

(3906, 671)

In [545]:
np.sqrt(((user_movie_rating.T - user_based_predict)** 2).mean())

0.09914256139416891

In [546]:
#item_based
movie_user_rating = user_movie_rating.T

movie_similarity = cosine_similarity(movie_user_rating)

temp2 = movie_similarity.dot(movie_user_rating)
item_based_predict = temp2/np.column_stack([movie_similarity.sum(axis=1)] * temp2.shape[-1])

In [547]:
item_based_predict.shape

(3906, 671)

In [548]:
movie_user_rating.shape

(3906, 671)

In [549]:
np.sqrt(((movie_user_rating - item_based_predict)** 2).mean())

0.09930055961868281

#cách k-neighbor

In [550]:
# Tạo một bản đồ ánh xạ từ movieId hiện tại sang các giá trị mới từ 0 đến 3905
movie_id_map = {movie_id: new_id for new_id, movie_id in enumerate(sorted(set(ratings['movieId'])))}

# Thay thế các giá trị trong cột 'movieId' của DataFrame với các giá trị mới từ 0 đến 3905
ratings['movieId'] = ratings['movieId'].map(movie_id_map)

In [551]:
ratings['userId'] -= 1

In [552]:
ratings

,userId,movieId,rating
5,0,690,2.0
10,0,718,2.5
11,0,738,1.0
13,0,1111,4.0
15,0,1171,2.0
...,...,...,...
99992,670,2107,4.0
99993,670,2131,3.5
99996,670,2146,4.5
99997,670,2149,4.0


In [553]:
rate_train, rate_test = train_test_split(ratings, test_size=0.1, random_state=40)

In [554]:
rate_train = rate_train.values.astype(int)
rate_test = rate_test.values.astype(int)

In [555]:
max(rate_train[:, 1])

3905

In [556]:
class CF(object):
    """docstring for CF"""
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1
    
    def normalize_Y(self):
        users = self.Y_data[:, 0] # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important 
        # for both memory and computing efficiency. For example, if #user = 1M, 
        # #item = 100k, then shape of the rating matrix would be (100k, 1M), 
        # you may not have enough memory to store this. Then, instead, we store 
        # nonzeros only, and, of course, their locations.
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        eps = 1e-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
        
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()
        
    
    def __pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: 
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others 
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:] 
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    
    def pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        if self.uuCF: return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)
    
    def recommend(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 
            

In [557]:
rs = CF(rate_train, k = 30, uuCF = 1)
rs.fit()
n_tests = rate_test.shape[0]

SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1], normalized = 0)
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print ('User-user CF, RMSE =', RMSE)

User-user CF, RMSE = 0.9348777140058424


In [558]:
rs = CF(rate_train, k = 30, uuCF = 0)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1], normalized = 0)
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print('Item-item CF, RMSE =', RMSE)

c:\Users\Dang Dung\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Dang Dung\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Item-item CF, RMSE = 1.0662162708372
